In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
import torchvision
import matplotlib.pyplot as plt
import dataset
from model import ResNet18YOLOv1
from loss import YOLOv1Loss
from tqdm.auto import tqdm

# About
This is an implementation of YOLOv1 from ***You Only Look Once: Unified, Real-Time Object Detection by Joseph Redmon, Santosh Divvala, Ross Girshick, and Ali Firhadi.*** Object detection is figuring out what objects are in an image and where they are. Another way to look at this problem is how can we write a computer program that draws bounding boxes around objects and predicts what kind of objects they are. YOLO solves this problem and does it super fast, like state of the art fast!

Let's talk about R-CNN, the predecessor to YOLO. It proposed regions, ran a classifier on every region, and did some post-processing to produce the final result. In simple language this translates to:
1. Lemme draw a lot of bounding boxes where I think objects are
2. Lemme figure out what are in the bounding boxes I drew
3. Ok, I drew too many bounding boxes, lemme remove most of them and keep the important ones

This is a lot of steps. What YOLO does instead is ***unified detection***. Unified detection combines the different components of object detection (where are the objects and what kind of objects are they) into one Convolutional Neural Network. You give it an image and in one swoop, it tells you exactly that.

Here's how it does it:
1. Divide the image into a SxS grid
2. Each cell in the grid predicts B bounding boxes and C class probabilities (what it thinks the object is)

We represent bounding boxes with 5 numbers: x, y, w, h, p.
- (x, y): center of the bounding box
- w: width
- h: height
- p: confidence (a measure of how confident we are that this box captures an object and matches the ground truth)

Accordingly, YOLOv1 produces a SxSx(5B+C) tensor. Each cell predicts B bounding boxes, how do we choose which one is the "true" predictor? How do we measure how good our bounding box and classification predictions are? 

We check which bounding box has the greatest overlap (IOU: Intersection Over Union) with the ground truth and choose that one as a predictor. We use this loss function to measure the "goodness" of our predictions:

![yolo loss function](https://i.stack.imgur.com/IddFu.png)

On a high level, it is the squared error between our prediction and the ground truth. 

# PASCAL VOC 2007 Dataset

PASCAL VOC Detection Dataset contains annotated images with 20 labelled classes and bounding boxes. There are 2,501 images in the training set, 2,510 images in the validation set, and 4,952 images in the test set.

In [2]:
# original dataset
pascal_voc_train = torchvision.datasets.VOCDetection(
    root="data",
    year="2007",
    image_set="train",
    download=False
)

pascal_voc_val = torchvision.datasets.VOCDetection(
    root="data",
    year="2007",
    image_set="val",
    download=False
)

pascal_voc_test = torchvision.datasets.VOCDetection(
    root="data",
    year="2007",
    image_set="test",
    download=False
)

In [3]:
# augment dataset for YOLOv1: resize and normalize image and convert bounding boxes from annotations to tensors
voc_train = dataset.PascalVOC(pascal_voc=pascal_voc_train)
voc_val = dataset.PascalVOC(pascal_voc=pascal_voc_val)
voc_test = dataset.PascalVOC(pascal_voc=pascal_voc_test)

TRANSFORMING PASCAL VOC
TRANSFORMING PASCAL VOC
TRANSFORMING PASCAL VOC


In [4]:
BATCH_SIZE = 64

In [5]:
# train_dataloader = DataLoader(voc_train, batch_size=BATCH_SIZE, shuffle=True)
# val_dataloader = DataLoader(voc_val, batch_size=BATCH_SIZE, shuffle=True)
# test_dataloader = DataLoader(voc_test, batch_size=BATCH_SIZE, shuffle=True)

train_dataloader = DataLoader(voc_train, batch_size=BATCH_SIZE, shuffle=False)
val_dataloader = DataLoader(voc_val, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(voc_test, batch_size=BATCH_SIZE, shuffle=False)

# Training

## Device

In [6]:
DEVICE = "cpu"

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    DEVICE = torch.device("mps")

DEVICE

device(type='mps')

## Hyperparameters
- S: dimensions of SxS grid
- B: number of bounding boxes predicted per cell
- C: number of classes

In [7]:
S = 7
B = 2
C = 20
lambda_coord = 5.0
lambda_noobj = 0.5

## Model
ResNet18 convolutional layers pretrained on ImageNet with 2 feedforward layers outputting a (N x S x S x (5B + C)) tensor.

In [81]:
yolo = ResNet18YOLOv1(S=S, B=B, C=C).to(DEVICE)

## Loss + Optimizer
![yolo loss function](https://i.stack.imgur.com/IddFu.png)

We use stochastic gradient descent with a learning rate of 1e-3, weight decay (L2 regularization) of 0.0005, and momentum of 0.9.

In [82]:
yolo_loss = YOLOv1Loss(S=S, B=B, C=C, lambda_coord=lambda_coord, lambda_noobj=lambda_noobj)

## Train
Train model with a learning rate of 1e-3 for the first few epochs, raise learning rate to 1e-2 and train for 75 epochs, then lower to 1e-3 for 30 epochs, and finally 1e-4 for 30 epochs.

We train the network for about 135 epochs.

In [83]:
def evaluate_loss(model, criterion, dataloader):
    total_loss = 0
    model.eval()
    
    with torch.no_grad():
        for X, Y in dataloader:
            X = X.to(DEVICE)
            Y = Y.to(DEVICE)

            pred = model(X)
            loss = criterion(pred, Y)
            total_loss += loss.item()
            break
            
    N = len(dataloader)
    # loss = total_loss / N
    loss = total_loss
    
    return loss

In [84]:
torch.cuda.empty_cache()

In [102]:
FIRST_EPOCHS = 3
EPOCHS = 100
MOMENTUM = 0.9
WEIGHT_DECAY = 0.0005

def train_yolo(model, criterion, train_dataloader, val_dataloader):
    pre_optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=WEIGHT_DECAY, momentum=MOMENTUM)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=WEIGHT_DECAY, momentum=MOMENTUM)
    # optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=WEIGHT_DECAY)
    # scheduler = MultiStepLR(optimizer,
    #                         milestones=[3, 6],
    #                         gamma=0.1)
    
    train_losses = []
    val_losses = []
    
    N = len(train_dataloader)
    
    # pretrain lr=1e-3
#     for epoch in range(FIRST_EPOCHS):
#         model.train()
#         total_loss = 0
#         for i, (X, Y) in enumerate(tqdm(train_dataloader, leave=False, desc=f"Pretrain Epoch [{epoch+1}/{FIRST_EPOCHS}]")):
#             X = X.to(DEVICE)
#             Y = Y.to(DEVICE)
            
#             pred = model(X)
#             loss = criterion(pred, Y)
#             total_loss += loss.item()
            
#             # backprop
#             pre_optimizer.zero_grad()
#             loss.backward()
#             pre_optimizer.step()
        
#         # loss = total_loss
#         loss = total_loss / N
#         print(f"Pretrain Epoch [{epoch+1}/{FIRST_EPOCHS}]: Train Loss={loss}")
            
    # train
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        lr = optimizer.param_groups[0]["lr"]
        for i, (X, Y) in enumerate(tqdm(train_dataloader, leave=False, desc=f"Epoch [{epoch+1}/{EPOCHS}]: lr={lr}")):
            X = X.to(DEVICE)
            Y = Y.to(DEVICE)
            
            # print(X)
            
            pred = model(X)
            
            # print(pred)
            loss = criterion(pred, Y)
            # print(loss)
            total_loss += loss.item()
            
            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            break
            
        # train_loss = total_loss / N
        train_loss = total_loss
        train_losses.append(train_loss)
        
        # update learning rate
        # scheduler.step()
        
        # evaluate on validation set
        val_loss = evaluate_loss(model, criterion, val_dataloader)
        val_losses.append(val_loss)
        
        print(f"Epoch [{epoch+1}/{EPOCHS}]: Train Loss={train_loss}, Val Loss={val_loss}")
    
    return train_losses, val_losses

In [103]:
train_losses, val_losses = train_yolo(yolo,
           yolo_loss,
           train_dataloader=train_dataloader,
           val_dataloader=val_dataloader)

Epoch [1/100]: lr=0.001:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch [1/100]: Train Loss=1.4226999282836914, Val Loss=9.295550346374512


Epoch [2/100]: lr=0.001:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch [2/100]: Train Loss=1.381443977355957, Val Loss=9.30438232421875


Epoch [3/100]: lr=0.001:   0%|          | 0/40 [00:00<?, ?it/s]

...

Epoch [98/100]: Train Loss=0.6955564618110657, Val Loss=9.373592376708984


Epoch [99/100]: lr=0.001:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch [99/100]: Train Loss=0.6922789812088013, Val Loss=9.37344741821289


Epoch [100/100]: lr=0.001:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch [100/100]: Train Loss=0.6350679397583008, Val Loss=9.38142204284668


In [ ]:
plt.plot(range(EPOCHS), train_losses)
plt.plot(range(EPOCHS), val_losses)